## Comparison of execution of VerticalPvConfiguration and DefaultConfiguration

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import pandas as pd

from agrivoltaics_supply_side_management.agriculture.crops import Cultivation
from agrivoltaics_supply_side_management.configuration \
    import BifacialVerticalPvConfiguration, DefaultBifacialConfiguration
from agrivoltaics_supply_side_management.photovoltaics.pv_modules\
    import BifacialElectricityGeneration
from agrivoltaics_supply_side_management.solar_irradiation.irradiance\
    import BifacialIrradianceManager

In [3]:
phi, alpha, theta, p_max = 0.055, 0.2, 0.96, 15

In [4]:
lattitude, longitude = 49.26757152616243, -123.25266177347093
timezone = 'Canada/Pacific'
time_range = pd.date_range('2022-07-06', '2022-07-07', freq='1T',
                             tz=timezone)

In [5]:
surface_azimuth = 180
surface_tilt = 20
axis_azimuth = 180
pvrow_height = 1
pvrow_width = 4
pitch = 10
gcr = pvrow_width / pitch
albedo = 0.2
n_pvrows = 3
index_observed_pvrow = 1
bifaciality = 0.75

In [6]:
bifacial_irradiance_manager = BifacialIrradianceManager(
    lattitude, longitude, timezone, time_range,
    surface_azimuth, surface_tilt, axis_azimuth, gcr,
    pvrow_height, pvrow_width, albedo, n_pvrows, index_observed_pvrow,
    bifaciality)

/Users/tadatoshi/Documents/development/projects/tadatoshi_solar_research_society/agrivoltaics_supply_side_management/venv/lib/python3.10/site-packages/pvfactors/geometry/base.py:349: ShapelyDeprecationWarning: Setting custom attributes on geometry objects is deprecated, and will raise an AttributeError in Shapely 2.0
  self.list_surfaces = list_surfaces
/Users/tadatoshi/Documents/development/projects/tadatoshi_solar_research_society/agrivoltaics_supply_side_management/venv/lib/python3.10/site-packages/pvfactors/geometry/base.py:350: ShapelyDeprecationWarning: Setting custom attributes on geometry objects is deprecated, and will raise an AttributeError in Shapely 2.0
  self.shaded = self._get_shading(shaded)
/Users/tadatoshi/Documents/development/projects/tadatoshi_solar_research_society/agrivoltaics_supply_side_management/venv/lib/python3.10/site-packages/pvfactors/geometry/base.py:351: ShapelyDeprecationWarning: Setting custom attributes on geometry objects is deprecated, and will rai

In [7]:
temp_model_parameters_type, module_name, inverter_name = (
    'open_rack_glass_glass', 'Trina_Solar_TSM_300DEG5C_07_II_',
    'ABB__MICRO_0_25_I_OUTD_US_208__208V_')

In [8]:
bifacial_electricity_generation = BifacialElectricityGeneration(
    lattitude, longitude, timezone,
    bifacial_irradiance_manager.bifacial_irradiances,
    temp_model_parameters_type,
    module_name, inverter_name,
    surface_tilt, surface_azimuth)

In [9]:
harvest_index = 0.95
biomass_energy_ratio = 30
leaf_area_index = 5
crop_growth_regulating_factor = 0.95
cultivation = Cultivation(harvest_index, biomass_energy_ratio,
                          leaf_area_index, crop_growth_regulating_factor)

In [10]:
net_photosynthetic_rate_parameters = {
                "phi": phi,
                "alpha": alpha,
                "theta": theta,
                "p_max": p_max
            }

In [11]:
ppfd_data = np.linspace(0, 1200)

In [12]:
bifacial_vertical_pv_configuration = BifacialVerticalPvConfiguration(
    net_photosynthetic_rate_parameters, ppfd_data,
    bifacial_irradiance_manager, None, 
    bifacial_electricity_generation,
    cultivation)

In [13]:
electricity_supply, crop_yield,\
    cumulative_electric_power_for_morning_peak, \
    cumulative_electric_power_for_afternoon_peak \
        = bifacial_vertical_pv_configuration.supply(time_range)

/Users/tadatoshi/Documents/development/projects/tadatoshi_solar_research_society/agrivoltaics_supply_side_management/venv/lib/python3.10/site-packages/pvlib/tools.py:340: RuntimeWarning: divide by zero encountered in true_divide
  np.trunc(np.log(atol / (df['VH'] - df['VL'])) / np.log(phim1)))


In [14]:
electricity_supply

2222.392542730113

In [15]:
crop_yield

0.0014309542561318596

In [16]:
cumulative_electric_power_for_morning_peak

287.4827210603035

In [17]:
cumulative_electric_power_for_afternoon_peak

548.2925089959637

In [19]:
default_bifacial_configuration = DefaultBifacialConfiguration(
    net_photosynthetic_rate_parameters, ppfd_data,
    bifacial_irradiance_manager, None, 
    bifacial_electricity_generation,
    cultivation)

In [20]:
default_electricity_supply, default_crop_yield, \
    default_cumulative_electric_power_for_morning_peak, \
    default_cumulative_electric_power_for_afternoon_peak \
        = default_bifacial_configuration.supply(time_range)

In [21]:
default_electricity_supply

2222.392542730113

In [22]:
default_crop_yield

0.0018059303109211973

In [23]:
default_cumulative_electric_power_for_morning_peak

287.4827210603035

In [24]:
default_cumulative_electric_power_for_afternoon_peak

548.2925089959637